<a href="https://colab.research.google.com/github/JahanzaibTayyab/AI-201/blob/main/Project_2_LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG-based QA System using LangChain, Pinecone, and Google Generative AI

This project notebook demonstrates the creation of a **Retrieval-Augmented Generation (RAG)**-based Question Answering (QA) system. It leverages powerful tools and APIs such as **LangChain**, **Pinecone**, and **Google Generative AI (Gemini Model)** to provide contextual answers to user queries based on uploaded documents. The notebook showcases a step-by-step process, from loading and splitting documents to creating embeddings, uploading data to Pinecone, and querying the system for responses.

## Key Features
1. **Pinecone for Vector Storage**  
   The project uses Pinecone as the vector database to store and retrieve document embeddings efficiently.  
   
2. **LangChain Integration**  
   LangChain simplifies document processing by handling text splitting, embedding generation, and connecting various AI components.  
   
3. **Google Generative AI (Gemini)**  
   Utilizes the Gemini 1.5 model for both embedding generation and language model-based reasoning to deliver intelligent and context-aware responses.  

4. **RAG Approach**  
   Combines retrieval and generation by searching for the most relevant document chunks and using an LLM to formulate coherent answers.  

5. **Flexible Querying**  
   Users can input any query, and the system retrieves the most relevant information from the indexed documents to provide accurate answers.  

## Workflow
1. **Setup**  
   Install and configure the required libraries such as LangChain, Pinecone Client, Google Generative AI, and TQDM.  

2. **Embedding Generation**  
   Process documents into chunks and generate embeddings using Google Generative AI's embedding model.  

3. **Vector Indexing**  
   Store the generated embeddings in a Pinecone index for efficient similarity-based retrieval.  

4. **Query Handling**  
   User queries are processed through a retrieval-augmented chain that fetches the most relevant documents, formulates a response, and displays the result.

5. **Source Document Traceability**  
   Returns the source documents used in formulating each response for transparency and validation.

## Use Cases
- Building document-based question-answering systems.
- Enhancing productivity by allowing quick access to relevant information from large datasets.
- Supporting tasks in education, research, and knowledge management.

## Technologies Used
- **LangChain**: Framework for building AI-powered applications.
- **Pinecone**: Vector database for storing and retrieving embeddings.
- **Google Generative AI (Gemini Model)**: Advanced generative AI for embedding and LLM capabilities.
- **Python**: Programming language for integrating APIs and data processing.

This project highlights the potential of combining vector search with generative AI to enable sophisticated knowledge retrieval systems.


In [ ]:
!pip install langchain
!pip install tqdm
!pip install langchain-pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10


In [ ]:
from google.colab import userdata
import os
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
PINECONE_ENVIRONMENT = 'us-east-1'

In [ ]:
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(
    api_key=PINECONE_API_KEY
)

# Check if the index exists; if not, create it
index_name = "anewindex"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",  # Choose the metric: cosine, euclidean, or dotproduct
        spec=ServerlessSpec(
            cloud="aws",
            region=PINECONE_ENVIRONMENT  # Use your environment's region
        )
    )

# # Connect to the index
index = pc.Index(name=index_name)
print(f"Successfully connected to index: {index_name}")


Successfully connected to index: anewindex


In [ ]:
!pip install -q -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 9.1 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY # Assuming PINECONE_API_KEY is already defined


embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Specify the desired embedding model
    api_key=GOOGLE_API_KEY
)

In [ ]:
!pip install -q -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("/content/document.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [ ]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)

    # Modify the upsert to use a dictionary for metadata
    index.upsert([{
        "id": doc.metadata["source"],  # Use "id" instead of the first tuple element
        "values": vector,  # Use "values" for the vector
        "metadata": {
            "text": doc.page_content,  # Add the text as part of metadata
            "source": doc.metadata["source"]  # Include the source
        }
    }])

100%|██████████| 29/29 [00:27<00:00,  1.07it/s]


In [ ]:
from langchain.vectorstores import Pinecone

# Use from_existing_index to load from an existing index
retriever = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings, text_key="text")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY,model="gemini-1.5-flash", temperature=0.7)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA

# Create a vector store using the Pinecone index
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)

# Create the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})  # Retrieve top 2 most similar documents

# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",
    retriever=retriever,  # Pass the retriever here
    return_source_documents=True  # Optional: to get the source documents used in the response
)

In [ ]:
query = "What is the Future of Agentic AI?"
print('Human Message:',query)
response = qa_chain.invoke(query)

# Print the answer
print("Agent Message:", response['result'])

# Print the source documents
# print("\n According To Given Information:")
for doc in response['source_documents']:
    print(f"- {doc.page_content[:200]}...")  # Print first 200 characters of each source document

In [ ]:
query = "What is the Future of AI Agents?"
print('Human Message:',query)
response = qa_chain.invoke(query)

# Print the answer
print("Agent Message:", response['result'])

# Print the source documents
print("\n According To Given Information:")
for doc in response['source_documents']:
    print(f"- {doc.page_content[:200]}...")  # Print first 200 characters of each source document